# with Pandas

In [3]:
import xml.etree.ElementTree as ET
import pandas as pd
import os
# Define the directory path containing RDL files
rdl_directory = r"C:\Users\sunny.chandel\Desktop\RDL_27"

# Define the namespace
namespace = '{http://schemas.microsoft.com/sqlserver/reporting/2016/01/reportdefinition}'

# Create a list to hold exception information
exceptions_info = []

# Initialize lists for datasets and parameters information
datasets_info = []
parameters_info = []

# Iterate through each RDL file in the directory
for filename in os.listdir(rdl_directory):
    if filename.endswith(".rdl"):
        # Load the XML file
        rdl_path = os.path.join(rdl_directory, filename)
        try:
            print(rdl_path)
            tree = ET.parse(rdl_path)
            root = tree.getroot()
            print(tree)

            # Extract report name from the file name (removing .rdl extension)
            report_name = os.path.splitext(filename)[0]
            # Extract the <Code> element from the root
            code_element = root.find(f'{namespace}Code')
            code_text = code_element.text.strip() if code_element is not None else None

            # Find the <DataSets> element using the namespace
            datasets_element = root.find(f'{namespace}DataSets')

            # Iterate through each <Dataset> element within <DataSets>
            for dataset_element in datasets_element:
                dataset_name = dataset_element.get('Name')  # Extract dataset name from the 'Name' attribute

                data_source_element = dataset_element.find(f'{namespace}Query/{namespace}DataSourceName')
                data_source = data_source_element.text if data_source_element is not None else None

                query_element = dataset_element.find(f'{namespace}Query/{namespace}CommandText')
                query = query_element.text if query_element is not None else None

                if query and query.startswith('='):
                    query = "'" + query

                fields_element = dataset_element.find(f'{namespace}Fields')

                # Gather information from each <Field> element within <Fields> for the current dataset
                for field_element in fields_element:
                    field_name = field_element.get('Name')
                    data_field_element = field_element.find(f'{namespace}DataField')
                    data_field = data_field_element.text if data_field_element is not None else None

                    datasets_info.append({
                        'Report Name': report_name,
                        'Dataset Name': dataset_name,
                        'Data Source': data_source,
                        'Query': query,
                        'Field Name': field_name,
                        'Data Field': data_field,
                        'Code': code_text
                    })

            # Find the <ReportParameters> element using the namespace
            parameters_element = root.find(f'{namespace}ReportParameters')
            if parameters_element is not None:
                # Iterate through each <ReportParameter> element within <ReportParameters>
                for parameter_element in parameters_element:
                    try:
                        parameter_name = parameter_element.get('Name')

                        parameter_data_type_element = parameter_element.find(f'{namespace}DataType')
                        if parameter_data_type_element is not None:
                            parameter_data_type = parameter_data_type_element.text
                        else:
                            parameter_data_type = None

                        parameter_default_value_element = parameter_element.find(f'{namespace}DefaultValue/{namespace}Values/{namespace}Value')
                        if parameter_default_value_element is not None:
                            parameter_default_value = parameter_default_value_element.text
                        else:
                            parameter_default_value = None

                        parameter_prompt_element = parameter_element.find(f'{namespace}Prompt')
                        if parameter_prompt_element is not None:
                            parameter_prompt = parameter_prompt_element.text
                        else:
                            parameter_prompt = None

                        valid_values_element = parameter_element.find(f'{namespace}ValidValues')
                        if valid_values_element is not None:
                            valid_values = []
                            for value_element in valid_values_element:
                                if value_element.tag == f'{namespace}ParameterValues':
                                    for sub_value_element in value_element:
                                        value = sub_value_element.find(f'{namespace}Value').text
                                        label = sub_value_element.find(f'{namespace}Label').text
                                        valid_values.append({'Value': value, 'Label': label})
                                elif value_element.tag == f'{namespace}DataSetReference':
                                    dataset_name = value_element.find(f'{namespace}DataSetName').text
                                    value_field = value_element.find(f'{namespace}ValueField').text
                                    label_field = value_element.find(f'{namespace}LabelField').text
                                    # Handle DataSetReference case accordingly
                                elif value_element.tag == f'{namespace}Query':
                                    # Handle Query case
                                    pass
                                elif value_element.tag == f'{namespace}StaticValues':
                                    # Handle StaticValues case
                                    pass
                                elif value_element.tag == f'{namespace}DynamicValues':
                                    # Handle DynamicValues case
                                    pass
                                else:
                                    # Handle other cases as needed
                                    pass

                            parameters_info.append({
                                'Report Name': report_name,
                                'Parameter Name': parameter_name,
                                'Data Type': parameter_data_type,
                                'Default Value': parameter_default_value,
                                'Prompt': parameter_prompt,
                                'Valid Values': valid_values
                            })
                        else:
                            parameters_info.append({
                                'Report Name': report_name,
                                'Parameter Name': parameter_name,
                                'Data Type': parameter_data_type,
                                'Default Value': parameter_default_value,
                                'Prompt': parameter_prompt,
                                'Valid Values': None
                            })

                    except Exception as e:
                        print(e)  # Print exception details for debugging
                        exceptions_info.append({
                            'Report Name': report_name,
                            'Parameter Name': parameter_name,
                            'Exception Details': str(e)
                        })
                else:
                    parameters_info.append({
                                'Report Name': report_name,
                                'Parameter Name': None,
                                'Data Type': None,
                                'Default Value': None,
                                'Prompt': None,
                                'Valid Values': None
                            })
                    
                        

        except Exception as e:
            exceptions_info.append({
                'File Name': filename,
                'Exception Details': str(e)
            })

# Create DataFrames from the lists
exceptions_df = pd.DataFrame(exceptions_info)
df_datasets = pd.DataFrame(datasets_info)
df_parameters = pd.DataFrame(parameters_info)

# Define the Excel writer
excel_writer = pd.ExcelWriter(r"C:\Users\sunny.chandel\Desktop\RDL_27\output1.xlsx", engine='xlsxwriter')

# Write the DataFrames to separate sheets
df_datasets.to_excel(excel_writer, sheet_name='Datasets', index=False)
df_parameters.to_excel(excel_writer, sheet_name='Parameters', index=False)
exceptions_df.to_excel(excel_writer, sheet_name='Exceptions', index=False)

# Save the Excel file
excel_writer.save()
print('done!!!')



C:\Users\sunny.chandel\Desktop\RDL_27\01 Focus Item Gap Analysis- Order-Sale Eligibilty Take Rate (Zone).rdl
C:\Users\sunny.chandel\Desktop\RDL_27\05 Product Analysis Focus Items - Sales - Order.rdl
C:\Users\sunny.chandel\Desktop\RDL_27\1 Week Category Sales Availability by Hour.rdl
C:\Users\sunny.chandel\Desktop\RDL_27\1 Week Category Sales Availability by Hour_Monday.rdl
C:\Users\sunny.chandel\Desktop\RDL_27\ABC Item Rank_SLIN.rdl
C:\Users\sunny.chandel\Desktop\RDL_27\All Wings Focus items.rdl
C:\Users\sunny.chandel\Desktop\RDL_27\Business System Results_ByZone.rdl
C:\Users\sunny.chandel\Desktop\RDL_27\Business System Results_Warabeya - Copy (2).rdl
C:\Users\sunny.chandel\Desktop\RDL_27\Business System Results_Warabeya - Copy.rdl
C:\Users\sunny.chandel\Desktop\RDL_27\Business System Results_Warabeya.rdl
C:\Users\sunny.chandel\Desktop\RDL_27\Focus Item-Rolling 14days.rdl
C:\Users\sunny.chandel\Desktop\RDL_27\Hot Food Flag - Monthly-CSV.rdl
C:\Users\sunny.chandel\Desktop\RDL_27\Peer Re

C:\Users\sunny.chandel\AppData\Local\Temp\ipykernel_39420\2343893629.py:178: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  excel_writer.save()


# Without Pandas


In [ ]:
import os
import xml.etree.ElementTree as ET
import xlsxwriter

# Define the directory path containing RDL files
rdl_directory = r"\\mstw-bitirs-x02\SSRS RDL\sample"

# Define the namespace
namespace = '{http://schemas.microsoft.com/sqlserver/reporting/2016/01/reportdefinition}'

# Create lists to hold information
datasets_info = []
parameters_info = []
exceptions_info = []
count=1
# Iterate through each RDL file in the directory
for filename in os.listdir(rdl_directory):
    if filename.endswith(".rdl"):
        rdl_path = os.path.join(rdl_directory, filename)
        print(count)
        count+=1
        try:
            tree = ET.parse(rdl_path)
            root = tree.getroot()

            # Extract report name from the file name (removing .rdl extension)
            report_name = os.path.splitext(filename)[0]

            # Extract the <Code> element from the root
            code_element = root.find(f'{namespace}Code')
            code_text = code_element.text.strip() if code_element is not None else None

            # Process datasets
            datasets_element = root.find(f'{namespace}DataSets')
            for dataset_element in datasets_element:
                dataset_name = dataset_element.get('Name')
                data_source_element = dataset_element.find(f'{namespace}Query/{namespace}DataSourceName')
                data_source = data_source_element.text if data_source_element is not None else None
                query_element = dataset_element.find(f'{namespace}Query/{namespace}CommandText')
                query = query_element.text if query_element is not None else None

                if query and query.startswith('='):
                    query = "'" + query

                fields_element = dataset_element.find(f'{namespace}Fields')
                for field_element in fields_element:
                    field_name = field_element.get('Name')
                    data_field_element = field_element.find(f'{namespace}DataField')
                    data_field = data_field_element.text if data_field_element is not None else None

                    datasets_info.append({
                        'Report Name': report_name,
                        'Dataset Name': dataset_name,
                        'Data Source': data_source,
                        'Query': query,
                        'Field Name': field_name,
                        'Data Field': data_field,
                        'Code': code_text
                    })

            # Process parameters
            parameters_element = root.find(f'{namespace}ReportParameters')
            if parameters_element is not None:
                for parameter_element in parameters_element:
                    parameter_name = parameter_element.get('Name')
                    parameter_data_type_element = parameter_element.find(f'{namespace}DataType')
                    parameter_data_type = parameter_data_type_element.text if parameter_data_type_element is not None else None
                    parameter_default_value_element = parameter_element.find(f'{namespace}DefaultValue/{namespace}Values/{namespace}Value')
                    parameter_default_value = parameter_default_value_element.text if parameter_default_value_element is not None else None
                    parameter_prompt_element = parameter_element.find(f'{namespace}Prompt')
                    parameter_prompt = parameter_prompt_element.text if parameter_prompt_element is not None else None
                    valid_values_element = parameter_element.find(f'{namespace}ValidValues')
                    if valid_values_element is not None:
                        valid_values = []
                        for value_element in valid_values_element:
                            if value_element.tag == f'{namespace}ParameterValues':
                                for sub_value_element in value_element:
                                    value = sub_value_element.find(f'{namespace}Value').text
                                    label = sub_value_element.find(f'{namespace}Label').text
                                    valid_values.append({'Value': value, 'Label': label})
                            elif value_element.tag == f'{namespace}DataSetReference':
                                dataset_name = value_element.find(f'{namespace}DataSetName').text
                                value_field = value_element.find(f'{namespace}ValueField').text
                                label_field = value_element.find(f'{namespace}LabelField').text
                                # Handle DataSetReference case accordingly
                            elif value_element.tag == f'{namespace}Query':
                                # Handle Query case
                                pass
                            elif value_element.tag == f'{namespace}StaticValues':
                                # Handle StaticValues case
                                pass
                            elif value_element.tag == f'{namespace}DynamicValues':
                                # Handle DynamicValues case
                                pass
                            else:
                                # Handle other cases as needed
                                pass
                    # Process other parameter-related data here...

                    parameters_info.append({
                        'Report Name': report_name,
                        'Parameter Name': parameter_name,
                        'Data Type': parameter_data_type,
                        'Default Value': parameter_default_value,
                        'Prompt': parameter_prompt,
                        'Valid Values': valid_values  # Replace with the appropriate data
                    })

        except Exception as e:
            exceptions_info.append({
                'File Name': filename,
                'Exception Details': str(e)
            })

# Create a new Excel workbook and add worksheets
output_path = r"C:\Users\scha3162\Downloads\PBIT_Files_unziped\output1.xlsx"
workbook = xlsxwriter.Workbook(output_path)
datasets_sheet = workbook.add_worksheet('Datasets')
parameters_sheet = workbook.add_worksheet('Parameters')
exceptions_sheet = workbook.add_worksheet('Exceptions')

# Write headers to the worksheets
datasets_headers = ['Report Name', 'Dataset Name', 'Data Source', 'Query', 'Field Name', 'Data Field', 'Code']
parameters_headers = ['Report Name', 'Parameter Name', 'Data Type', 'Default Value', 'Prompt', 'Valid Values']
exceptions_headers = ['File Name', 'Exception Details']

datasets_sheet.write_row(0, 0, datasets_headers)
parameters_sheet.write_row(0, 0, parameters_headers)
exceptions_sheet.write_row(0, 0, exceptions_headers)

# Write data to the worksheets
for row, dataset in enumerate(datasets_info, start=1):
    datasets_sheet.write_row(row, 0, [dataset[field] for field in datasets_headers])

for row, parameter in enumerate(parameters_info, start=1):
    valid_values_formatted = ', '.join([f"{value['Value']}: {value['Label']}" for value in parameter['Valid Values']])
    parameters_sheet.write_row(row, 0, [parameter[field] for field in parameters_headers[:-1]] + [valid_values_formatted])

for row, exception in enumerate(exceptions_info, start=1):
    exceptions_sheet.write_row(row, 0, [exception[field] for field in exceptions_headers])

# Close the workbook
workbook.close()

print('Done!')
